<font size="6"><i>This code illustrates the implementation of Bayesian Statistics to find the contribution of marketing channels on sales for the dataset present here [MMM Data Excel](https://github.com/ankursikka/Impact-DS/raw/main/Market-Mix-Model-(MMM)/Bayesian/MMM_data.xlsx)</b>.</i></font>

<hr>

In [25]:
# Importing necessary modules

import pandas as pd
import numpy as np
import pymc as pm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

%matplotlib inline

# Setting display options

pd.set_option('display.width',1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 800)
pd.set_option('display.float_format', '{:.2f}'.format)

In [39]:
# Import the data

df = pd.read_excel('https://github.com/ankursikka/Impact-DS/raw/main/Market-Mix-Model-(MMM)/Bayesian/MMM_data.xlsx',sheet_name='Sheet1')

In [3]:
# Let us look at the top rows

df.head()

,TV Manufacturing Brand,DATE,DEMAND,Consumer Price Index (CPI),Consumer Confidence Index(CCI),Producer Price Index (PPI),Unit Price ($),POS/ Supply Data,SALES ($),Advertising Expenses (SMS),Advertising Expenses(Newspaper ads),Advertising Expenses(Radio),Advertising Expenses(TV),Advertising Expenses(Internet),GRP (NewPaper ads),GRP(SMS),GRP(Radio,GRP(Internet),GRP(TV)
0,TV Manufacturing & Supplier Unit,2010-01-01,4384,104.9,96.3,106.7,361.62,4240,1533268.80,77.4819,14.104193,112.3370,1479.4565,722.571,95.333,11.8398,91.0000,276.3636,756.5909
1,TV Manufacturing & Supplier Unit,2010-01-02,4366,104.9,96.3,106.7,361.62,4266,1542670.92,73.4783,13.298758,105.7133,1369.8913,717.857,114.957,27.8039,111.9091,291.3182,860.1364
2,TV Manufacturing & Supplier Unit,2010-01-03,4006,104.9,96.3,106.7,361.62,4206,1520973.72,80.6093,13.200691,108.7702,1428.0645,653.333,113.090,0.0000,94.6364,282.7273,751.9545
3,TV Manufacturing & Supplier Unit,2010-01-04,4076,104.9,96.3,106.7,361.62,4176,1510125.12,65.9319,12.721429,93.3065,1309.3548,622.095,72.442,0.0000,98.5909,306.3182,749.4545
4,TV Manufacturing & Supplier Unit,2010-01-05,4834,104.9,96.3,106.7,361.62,5234,1892719.08,77.4819,14.104193,112.3370,1479.4565,722.571,95.333,11.8398,91.0000,276.3636,756.5909


<i><b>Let us do some exploratory data analysis and see if there are any issues or data gaps</b> </i>

In [4]:
# Checking total number of unique values

print(df.nunique(axis=0).sort_values().to_string())

TV Manufacturing Brand                    1
Unit Price ($)                            6
Producer Price Index (PPI)               11
Consumer Confidence Index(CCI)           20
Consumer Price Index (CPI)               22
GRP(SMS)                                 74
Advertising Expenses(Internet)          100
GRP(Radio                               101
Advertising Expenses(Radio)             101
Advertising Expenses(TV)                102
Advertising Expenses (SMS)              102
GRP(Internet)                           102
Advertising Expenses(Newspaper ads)     102
GRP (NewPaper ads)                      103
GRP(TV)                                 103
POS/ Supply Data                        539
DEMAND                                  816
SALES ($)                               950
DATE                                   2613


In [8]:
# Checking column types

df.dtypes

TV Manufacturing Brand                         object
DATE                                   datetime64[ns]
DEMAND                                          int64
Consumer Price Index (CPI)                    float64
Consumer Confidence Index(CCI)                float64
Producer Price Index (PPI)                    float64
Unit Price ($)                                float64
POS/ Supply Data                                int64
SALES ($)                                     float64
Advertising Expenses (SMS)                    float64
Advertising Expenses(Newspaper ads)           float64
Advertising Expenses(Radio)                   float64
Advertising Expenses(TV)                      float64
Advertising Expenses(Internet)                float64
GRP (NewPaper ads)                            float64
GRP(SMS)                                      float64
GRP(Radio                                     float64
GRP(Internet)                                 float64
GRP(TV)                     

<i><b>Data appears to be loaded in correct format and does not require explicit transformation to numeric or date field</b> </i>

In [16]:
# Check for nulls

df.isna().any()

TV Manufacturing Brand                 False
DATE                                   False
DEMAND                                 False
Consumer Price Index (CPI)             False
Consumer Confidence Index(CCI)         False
Producer Price Index (PPI)             False
Unit Price ($)                         False
POS/ Supply Data                       False
SALES ($)                              False
Advertising Expenses (SMS)             False
Advertising Expenses(Newspaper ads)    False
Advertising Expenses(Radio)            False
Advertising Expenses(TV)               False
Advertising Expenses(Internet)         False
GRP (NewPaper ads)                     False
GRP(SMS)                               False
GRP(Radio                              False
GRP(Internet)                          False
GRP(TV)                                False
dtype: bool

<i><b>None of the fields have null values so we don't need to fix that</b> </i>

In [17]:
# Describing the data

df.describe()

,DEMAND,Consumer Price Index (CPI),Consumer Confidence Index(CCI),Producer Price Index (PPI),Unit Price ($),POS/ Supply Data,SALES ($),Advertising Expenses (SMS),Advertising Expenses(Newspaper ads),Advertising Expenses(Radio),Advertising Expenses(TV),Advertising Expenses(Internet),GRP (NewPaper ads),GRP(SMS),GRP(Radio,GRP(Internet),GRP(TV)
count,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00,2613.00
mean,5021.43,102.61,103.15,102.24,363.28,4522.97,1641506.68,60.39,12.65,88.07,1324.50,3079.18,505.30,30.62,139.44,286.23,1146.11
std,2681.20,1.38,3.17,2.04,26.37,2604.00,941667.28,13.56,1.12,12.57,123.68,1520.89,488.15,31.57,146.04,138.41,822.99
min,1610.00,101.30,96.30,99.50,282.14,1510.00,462709.60,37.92,10.03,62.97,1067.16,0.00,5.66,0.00,66.86,191.18,697.64
25%,2436.00,101.40,102.70,100.40,361.60,1776.00,671767.90,47.55,11.89,78.10,1251.25,2226.43,114.96,0.00,95.14,234.95,849.55
50%,4636.00,102.30,103.60,102.70,361.62,4412.00,1605094.92,61.27,13.19,84.13,1380.70,3302.67,221.53,26.12,109.64,261.45,928.55
75%,6834.00,103.40,104.60,103.50,361.62,6266.00,2267205.60,71.01,13.44,98.64,1416.17,4237.10,854.31,46.78,126.05,291.32,1114.86
max,18565.00,106.50,107.90,107.20,400.10,16482.00,5960220.84,89.73,14.10,118.47,1479.46,6354.57,1791.18,145.00,1169.41,1540.43,7307.32


<i><b>There are a couple of fields with 0 values and large standard deviation. First, we will adjust the column names to be more python friendly. Post that we will visualize the data.</b> </i>

In [47]:
# Adjusting the column names

df.columns = df.columns.str.replace("[\s)/]",'', regex=True).str.replace("(",'_', regex=False)
df.columns

Index(['TVManufacturingBrand', 'DATE', 'DEMAND', 'ConsumerPriceIndex_CPI', 'ConsumerConfidenceIndex_CCI', 'ProducerPriceIndex_PPI', 'UnitPrice_$', 'POSSupplyData', 'SALES_$', 'AdvertisingExpenses_SMS', 'AdvertisingExpenses_Newspaperads', 'AdvertisingExpenses_Radio', 'AdvertisingExpenses_TV', 'AdvertisingExpenses_Internet', 'GRP_NewPaperads', 'GRP_SMS', 'GRP_Radio', 'GRP_Internet', 'GRP_TV'], dtype='object')

In [50]:
# sns.displot(df['Advertising Expenses(Internet)'])

plt.figure(figsize=(20,6))
fig = px.line(data_frame=df, x='DATE',y='AdvertisingExpenses_Internet')
fig.show()

<Figure size 2000x600 with 0 Axes>

<center><font size=12><b>Under Construction!!!</b></font></center>